## Compression

In [3]:
class CompressedGene:
    def __init__(self, gene):
        self._compress(gene)
    
    def _compress(self, gene):
        self.bit_string = 1 # start with sentinel
        for nucleotide in gene.upper():
            self.bit_string <<= 2 # shift left two bits
            if nucleotide == "A": # change last two bits to 00
                self.bit_string |= 0b00
            elif nucleotide == "C": # change last two bits to 01
                self.bit_string |= 0b01
            elif nucleotide == "G": # change last two bits to 10
                self.bit_string |= 0b10
            elif nucleotide == "T": # change last two bits to 11
                self.bit_string |= 0b11
            else:
                raise ValueError("Invalid Nucleotide:{}".format(nucleotide))

    def decompress(self):
        gene = ""
        for i in range(0, self.bit_string.bit_length() - 1, 2): # - 1 to exclude sentinel
            bits = self.bit_string >> i & 0b11 # get just 2 relevant bits
            if bits == 0b00: # A
                gene += "A"
            elif bits == 0b01: # C
                gene += "C"
            elif bits == 0b10: # G
                gene += "G"
            elif bits == 0b11: # T
                gene += "T"
            else:
                raise ValueError("Invalid bits:{}".format(bits))
        return gene[::-1] # [::-1] reverses string by slicing backward

    def __str__(self) -> str: # string representation for pretty printing
        return self.decompress()

In [14]:
from sys import getsizeof

original = "TAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATA" * 100

print("original is {} bytes".format(getsizeof(original)))

compressed = CompressedGene(original) # compress
print("compressed is {} bytes".format(getsizeof(compressed.bit_string)))

print("original and decompressed are the same: {}".format(original == compressed.decompress()))

original is 8649 bytes
compressed is 2320 bytes
original and decompressed are the same: True


## unbreakable_encryption.py

In [81]:
from secrets import token_bytes

def random_key(length):
    
    # generate length random bytes
    tb = token_bytes(length)
    
    # convert those bytes into a bit string and return it
    return int.from_bytes(tb, "big")

def encrypt(original):
    original_bytes = original.encode()
    dummy = random_key(len(original_bytes))
    original_key = int.from_bytes(original_bytes, "big")
    encrypted = original_key ^ dummy # XOR
    return dummy, encrypted

def decrypt(dummy, encrypted):
    decrypted = dummy ^ encrypted # XOR
    temp = decrypted.to_bytes((decrypted.bit_length()+ 7) // 8, "big")
    return temp.decode()

In [82]:
message = 'emre'
key, script = encrypt(message)
print(script)
result = decrypt(key, script)
print(message)

1021470958
emre


## Calculating pi
π = 4/1 - 4/3 + 4/5 - 4/7 + 4/9 - 4/11...

In [86]:
def calculate_pi(n_terms):
    numerator = 4.0
    denominator = 1.0
    operation = 1.0
    pi = 0.0
    for _ in range(n_terms):
        pi += operation * (numerator / denominator)
        denominator += 2.0
        operation *= -1.0
    return pi

In [87]:
print(calculate_pi(1000000))

3.1415916535897743


## The Towers of Hanoi

In [137]:
class Stack():
    def __init__(self):
        self._container = []

    def push(self, item):
        self._container.append(item)

    def pop(self):
        return self._container.pop()

    def __repr__(self):
        return repr(self._container)

In [138]:
discs = ['Large', 'Medium', 'Small']
tower_a = Stack()
tower_b = Stack()
tower_c = Stack()
for i in discs:
    tower_a.push(i)

In [139]:
def hanoi(begin, end, temp, n):
    if n == 1:
        print('finish')
        end.push(begin.pop())
        print(begin, end, temp)
    else:
        print('x')
        hanoi(begin, temp, end, n - 1)
        print('y')
        hanoi(begin, end, temp, 1)
        print('z')
        hanoi(temp, end, begin, n - 1)

In [140]:
hanoi(tower_a, tower_c, tower_b, num_discs)
print(tower_a)
print(tower_b)
print(tower_c)

x
x
finish
['Large', 'Medium'] ['Small'] []
y
finish
['Large'] ['Medium'] ['Small']
z
finish
[] ['Medium', 'Small'] ['Large']
y
finish
[] ['Large'] ['Medium', 'Small']
z
x
finish
['Medium'] ['Small'] ['Large']
y
finish
[] ['Large', 'Medium'] ['Small']
z
finish
[] ['Large', 'Medium', 'Small'] []
[]
[]
['Large', 'Medium', 'Small']


## DNA Search 
Genes are commonly represented in computer software as a sequence of the characters
A, C, G, and T. Each letter represents a nucleotide, and the combination of
three nucleotides is called a codon. A codon codes
for a specific amino acid that together with other amino acids can form a protein.
A classic task in bioinformatics software is to find a particular codon within a gene.

In [142]:
from enum import IntEnum

Nucleotide = IntEnum('Nucleotide', ('A', 'C', 'G', 'T'))
print(Nucleotide)

<enum 'Nucleotide'>


In [144]:
#Codon = (Nucleotide, Nucleotide, Nucleotide) # type alias for codons
#Gene = [Codon] # type alias for genes

In [145]:
gene_str = "ACGTGGCTCTCTAACGTACGTACGTACGGGGTTTATATATACCCTAGGACTCCCTTT"

def string_to_gene(s):
    gene = []
    for i in range(0, len(s), 3):
        if (i + 2) >= len(s): # don't run off end!
            return gene
        
        # initialize codon out of three nucleotides
        codon = (Nucleotide[s[i]], Nucleotide[s[i + 1]], Nucleotide[s[i + 2]])
        gene.append(codon) # add codon to gene
    return gene

In [149]:
my_gene = string_to_gene(gene_str)
print(my_gene)

[(<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.G: 3>), (<Nucleotide.T: 4>, <Nucleotide.G: 3>, <Nucleotide.G: 3>), (<Nucleotide.C: 2>, <Nucleotide.T: 4>, <Nucleotide.C: 2>), (<Nucleotide.T: 4>, <Nucleotide.C: 2>, <Nucleotide.T: 4>), (<Nucleotide.A: 1>, <Nucleotide.A: 1>, <Nucleotide.C: 2>), (<Nucleotide.G: 3>, <Nucleotide.T: 4>, <Nucleotide.A: 1>), (<Nucleotide.C: 2>, <Nucleotide.G: 3>, <Nucleotide.T: 4>), (<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.G: 3>), (<Nucleotide.T: 4>, <Nucleotide.A: 1>, <Nucleotide.C: 2>), (<Nucleotide.G: 3>, <Nucleotide.G: 3>, <Nucleotide.G: 3>), (<Nucleotide.G: 3>, <Nucleotide.T: 4>, <Nucleotide.T: 4>), (<Nucleotide.T: 4>, <Nucleotide.A: 1>, <Nucleotide.T: 4>), (<Nucleotide.A: 1>, <Nucleotide.T: 4>, <Nucleotide.A: 1>), (<Nucleotide.T: 4>, <Nucleotide.A: 1>, <Nucleotide.C: 2>), (<Nucleotide.C: 2>, <Nucleotide.C: 2>, <Nucleotide.T: 4>), (<Nucleotide.A: 1>, <Nucleotide.G: 3>, <Nucleotide.G: 3>), (<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.T: 4

In [147]:
def linear_contains(gene, key_codon):
    for codon in gene:
        if codon == key_codon:
            return True
    return False

In [148]:
acg = (Nucleotide.A, Nucleotide.C, Nucleotide.G)
gat = (Nucleotide.G, Nucleotide.A, Nucleotide.T)
print(linear_contains(my_gene, acg)) # True
print(linear_contains(my_gene, gat)) # False

True
False


In [154]:
def binary_contains(gene, key_codon):
    low = 0
    high = len(gene) - 1
    while low <= high: # while there is still a search space
        mid = (low + high) // 2
        if gene[mid] < key_codon:
            low = mid + 1
        elif gene[mid] > key_codon:
            high = mid - 1
        else:
            return True
    return False

In [155]:
my_sorted_gene = sorted(my_gene)
print(binary_contains(my_sorted_gene, acg)) # True
print(binary_contains(my_sorted_gene, gat)) # False

True
False


In [156]:
print(binary_contains(["a", "d", "e", "f", "z"], "f")) # True
print(binary_contains(["john", "mark", "ronald", "sarah"], "sheila")) # False

True
False
